<h1><font color='blue'><center> Redes recurrentes y LSTM en Deep Learning </center></font></h1>

<h2>Aplicando redes neuronales recurrentes /LSTM para el modelado de lenguaje</h2>

Estudiaremos el modelado de lenguaje y crearemos una red neuronal recurrente basada en LSTM para entrenar y hacer un benchmarking sobre el dataset Penn Treebank.


<h2>Objetivo</h2>

Las redes neuronales recurrentes procesan datos secuenciales siguiendo el estado o contexto. Aquí estudiaremos el **modelado de lenguaje**, una tarea muy relevante en diferentes problemas linguísticos, como **reconocimiento de voz, traducción de máquinas y subtítulos de imágenes**. Usaremos el dataset Penn Treebank.


<h2>Tabla de contenido</h2>
<ol>
    <li><a href="#language_modelling">Qué es el modelado de lenguaje?</a></li>
    <li><a href="#treebank_dataset">Dataset Penn Treebank</a></li>
    <li><a href="#word_embedding">Word Embedding (Incrustación de palabras)</a></li>
    <li><a href="#building_lstm_model">Construyendo el modelo LSTM para el modelado de lenguaje</a></li>
    <li><a href="#ltsm">LTSM</a></li>
</ol>
<p></p>
</div>
<br>


* * *


<a id="language_modelling"></a>

<h2>Qué es el modelado de lenguaje?</h2>
El modelado de lenguaje, en términos simples, es la tarea de asignarle probabilidades a secuencias de palabras. Esto significa que, dado un contexto de una o una secuencia de palabras en el lenguaje que el modelo ha sido entrenado, el modelo debe brindar las palabras siguientes más probables. Es una de las tareas más importantes en el procesamiento natural del lenguaje.

<img src="https://ibm.box.com/shared/static/1d1i5gub6wljby2vani2vzxp0xsph702.png" width="1080">
<center><i>Ejemplo de una sentencia siendo predecida</i></center>
<br><br>
En este ejemplo, se pueden ver las predicciones para la siguiente palabra de una sentencia, dado el contexto "This is an". Puede verse que esto se reduce a una tarea de análisis secuencial -- se le da una palabra o sentencia de palabras (datos de entrada), y, dado el contexto (el estado), necesita encontrar la siguiente palabra (predicción).

<img src="https://ibm.box.com/shared/static/az39idf9ipfdpc5ugifpgxnydelhyf3i.png" width="1080">
<center><i>El ejemplo de arriba es un esquema de una RNN en ejecución</i></center>
<br><br>
Como se muestra en la imagen de arriba, las redes neuronales recurrentes encajan este problema como un guante. Junto con LSTM y su capacidad de mantener el estado del modelo por más de 1000 pasos temporales, tenemos todas las herramientas que necesitamos para emprender este problema. El objetivo es crear un modelo que pueda alcanzar altos niveles de perplejidad en el dataset deseado.

En problemas de modelado de lenguaje, la perplejidad (perplexity) es la forma de medir la eficiencia. La perplejidad es una medidad de qué tan bien un modelo probabilístico es capaz de predecir su muestra. Una forma de más alto nivel de explicar esto sería decir que una baja perplejidad significa un mayor grado de confianza en las predicciones que realiza el modelo. Así, mientras menor sea la perplejidad, mejor.



<a id="treebank_dataset"></a>

<h2>El dataset Penn Treebank</h2>
Históricamente, conseguir datasets lo suficientemente grandes para el procesado de lenguaje natural (Natural Language Processing NLP) ha sido complejo. Esto en parte a la necesidad de que las sentencias sean descompuestas y etiquetadas con un cierto grado de corrección. Esto significa que necesitamos un gran volumen de datos, anotados o al menos corregido por humanos, y esto no es una tarea simple.

El dataset Penn Treebank es un dataset mantenido por la universidad de Pensilvania. Tiene 4.800.000 palabras en él, todas corregidas por humanos. Está compuesto por múltiples fuentes. Debido a su tamaño y nivel de corrección es comúnmente utilizado para el benchmarking en problema de NLP.

El conjunto de datos se divide en diferentes tipos de anotaciones, como esqueletos de fragmentos de discurso, sintácticos y semánticos. Para este ejemplo, simplemente usaremos una muestra de palabras limpias y sin anotaciones (con la excepción de un tag -- --<code><unk></code>, que es utilizado para palabras raras tales como sustantivos poco comunes) para nuestro modelo. Esto significa que sólo queremos predecir la próxima palabra, no lo que quiere decir en el contexto o sus clases en una sentencia dada.


<center>Ejemplo de texto del dataset que utilizaremos: <b>ptb.train</b></center>
<br><br>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <center>the percentage of lung cancer deaths among the workers at the west <code>&lt;unk&gt;</code> mass. paper factory appears to be the highest for any asbestos workers studied in western industrialized countries he said 
 the plant which is owned by <code>&lt;unk&gt;</code> & <code>&lt;unk&gt;</code> co. was under contract with <code>&lt;unk&gt;</code> to make the cigarette filters 
 the finding probably will support those who argue that the U.S. should regulate the class of asbestos including <code>&lt;unk&gt;</code> more <code>&lt;unk&gt;</code> than the common kind of asbestos <code>&lt;unk&gt;</code> found in most schools and other buildings dr. <code>&lt;unk&gt;</code> said</center>
</div>


<a id="word_embedding"></a>

<h2>Word Embeddings</h2><br/>

Para un mejor procesamiento, en este ejemplo usaremos <a href="https://www.tensorflow.org/tutorials/word2vec/"><b>word embeddings</b></a>, que es una forma de representar estructuras de secuencias o palabras como vectores n-dimensionales (donde n es un número razonablemente alto, como 200 o 500). Básicamente, le asignaremos a cada palabra un vector inicializado aleatoriamente, e ingresaremos ellos a la red para ser procesados. Luego de cierto número de iteraciones, estos vectores se espera asuman valores que ayuden a la red a predecir correctamente lo que se necesita -- en nuestro caso, la siguiente palabra en la sentencia. Esta ha demostrado ser una tarea muy eficiente en el NLP.
<br><br>
<font size="4"><strong>
$$Vec("Example") = [0.02, 0.00, 0.00, 0.92, 0.30, \ldots]$$
</strong></font>
<br>
Word Embedding tiende a agrupar palabras que se usan de forma similar de manera razonable en un espacio vectorial. Por ejemplo, si usamos T-SNE (un algoritmo de reducción de dimensiones de visualización) para aplanar las dimensiones de nuestros vectores en un espacio 2-dimensional y graficamos estas palabras en un espacio 2-dimensional, podemos ver algo como esto:

<img src="https://ibm.box.com/shared/static/bqhc5dg879gcoabzhxra1w8rkg3od1cu.png" width="800">
<center><i>T-SNE Mockup with clusters marked for easier visualization</i></center>
<br><br>
Puede ver que por ejemplo "None" es bastante parecida semánticamente a "Zero", mientras que en una frase que use "Italy", dicha palabra podría intercambiarse por "Germany" con un daño muy pequeño a la estructura de la sentencia. La "cercanía" vectorial para palabras similares es un gran indicador de un modelo bien construido. 

<hr>
 


Debemos importar los módulos necesarios. <b><code>tensorflow.models.rnn</code></b> incluye la función para construir RNNs, y <b><code>tensorflow.models.rnn.ptb.reader</code></b> es un módulo ayudante para obtener los datos de entrada del dataset descargado.

Si desea aprender más puede echar un vistazo a <https://github.com/tensorflow/models/blob/master/tutorials/rnn/ptb/reader.py>


In [1]:
#!pip install tensorflow==2.2.0rc0
#!pip install numpy


In [3]:
import time
import numpy as np
import tensorflow as tf
if not tf.__version__ == '2.2.0-rc1': #2.2.0-rc0
    print(tf.__version__)
    raise ValueError('please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)')

In [9]:
!mkdir data
!mkdir data/ptb
!wget -q -O data/ptb/reader.py https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0120EN-SkillsNetwork/labs/Week3/data/ptb/reader.py
!cp data/ptb/reader.py . 



Archive:  data/ptb.zip
   creating: data/ptb/
  inflating: data/ptb/reader.py      
   creating: data/__MACOSX/
   creating: data/__MACOSX/ptb/
  inflating: data/__MACOSX/ptb/._reader.py  
  inflating: data/__MACOSX/._ptb     


In [6]:
import reader

2.2.0-rc0


<a id="building_lstm_model"></a>

<h2>Construyendo el modelo LSTM para el modelado de lenguaje</h2>
Debemos comenzar por extraer el dataset <code>simple-examples</code>.


In [10]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
!tar xzf simple-examples.tgz -C data/

--2020-08-31 00:12:47--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)...147.229.9.23
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80...connected.
HTTP request sent, awaiting response...200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz’

simple-examples.tgz 100%[===================>]  33.25M  58.2KB/s    in 8m 28s  

2020-08-31 00:21:16 (67.1 KB/s) - ‘simple-examples.tgz’ saved [34869662/34869662]



Para hacer más fácil jugar con los hiperparámetros del modelo, los declaramos de antemano.


In [2]:
#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size_l1 = 256
hidden_size_l2 = 128
#The maximum number of epochs trained with the initial learning rate
max_epoch_decay_lr = 4
#The total number of epochs in training
max_epoch = 15
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 30
#The size of our vocabulary
vocab_size = 10000
embeding_vector_size= 200
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "data/simple-examples/data/"

Algunas aclaraciones acerca de la arquitectura:


Estructura de la red:

<ul>
    <li> El número de celdas LSTM es 2. Para darle al modelo más poder expresivo, podemos agregar múltiples capas de LSTMs para procesar los datos.
    </li>
    <li> Los pasos de recurrencia son 20,esto es, cuando nuestra RNN está "desplegada" el paso de recurrencia es 20.</li>   
    <li>La estructura es:
        <ul>
            <li>200 unidades de entrada -> [200x200] pesos -> 200 unidades escondidas (primera capa) -> [200x200] matriz de pesos  -> 200 unidades escondidas (segunda capa) ->  [200] matriz de pesos -> 200 unidades de salida</li>
        </ul>
    </li>
</ul>
<br>

Capa de entrada: 

<ul>
    <li>La red tiene 200 unidades de entrada</li>
    <li> Supongamos que cada palabra es representada por un vector de incrustado de dimensión e=200. La capa de entrada de cada celda tendrá 200 unidades lineales. Estas e=200 unidades lineales están conectadas a cada una de las h=200 unidades LSTM en la capa oculta (asumiendo sólo hay una capa oculta, aunque nuestro caso tiene 2 capas)
    </li>
    <li>El tamaño de entrada es [batch_size, num_steps], esto es [30x20]. Se convertirá en [30x20x200] luego de la incrustación, y luego en 20x[30x200]
    </li>
</ul>
<br>

Capa oculta:

<ul>
    <li> Cada LSTM tiene 200 unidades ocultas, que es el equivalente a la dimensión de los embedding words y salida</li>
</ul>
<br>


Hay mucho por hacer y mucha información para proceasr al mismo tiempo.

El código está adaptado del ejemplo <a href="https://github.com/tensorflow/models">PTBModel</a>


<h3>Datos de entrenamiento</h3>
La historia comienza con los datos:
<ul>
    <li> Los datos de entrenamiento son una lista de palabras, de tamaño 929589, representadas por números, por ejemplo [9971, 9972, 9974, 9975,...]</li>
    <li> Leemos los datos en mini-lotes de tamaño b=30. Asumimos el tamaño de cada sentencia es de 20 palabras (num_steps=20). Luego, tomará $$floor(\frac{N}{b \times h})+1=1548$$ iteraciones para el aprendiz atravesar todas las sentencias una vez. N es el tamaño de la lista de palabras, b es el batch size (tamaño del lote) y h es el tamaño de cada sentencia. Así, el número de iteradores es 1548.
    </li>
    <li>Cada lote de datos es leído desde el dataset de entrenamiento de tamaño 600 y forma [30x20]</li>
</ul>


In [4]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, vocab, word_to_id = raw_data

In [5]:
len(train_data)

929589

In [6]:
def id_to_word(id_list):
    line = []
    for w in id_list:
        for word, wid in word_to_id.items():
            if wid == w:
                line.append(word)
    return line            
                

print(id_to_word(train_data[0:100]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', '<eos>', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', '<eos>', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', '<eos>', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', '<eos>', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of']


Leamos un mini-lote y alimentémoslo en la red:


In [7]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple = itera.__next__()
_input_data = first_touple[0]
_targets = first_touple[1]

In [8]:
_input_data.shape

(30, 20)

In [9]:
_targets.shape

(30, 20)

Miremos 3 sentencias de nuestra entrada x:


In [10]:
_input_data[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21]],
      dtype=int32)

In [11]:
print(id_to_word(_input_data[0,:]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim']


<h3>Embeddings</h3>

Debemos convertir las palabras en nuestro dataset a vectores de números. El enfoque tradicional es usar one-hot encoding, que habitualmente se usa para convertir valores categóricos en numéricos. Sin embargo, los vectores con esta codificación tienen muchas dimensiones y son computacionalmente ineficientes. Así que usaremos el enfoque word2vec. Es, de hecho, una capa en nuestra red LSTM, donde los IDs de palabras serán representados como una representación densa antes de alimentarlos en la red.

Los vectores embebidos son actualizados durante el proceso de entrenamiento.
Creamos las incrustaciones para nuestros datos de entrada. <b>embedding_vocab</b> es una matriz de   [10000x200] para todas las 10000 palabras únicas.


<b>embedding_lookup()</b> encuentra los valores embebidos para  nuestro lote de 30x20 palabras. Va a cada fila de input_data, y por cada palabra en la fila/sentencia, encuentra el vector correspondiente en embedding_dic.
Crea un tensor [30x20x200] , así que el primer elemento de inputs (la primer sentencia), es una matriz de 20x200, donde cada fila de ella, es un vector representando una palabra en la sentencia.


In [12]:
embedding_layer = tf.keras.layers.Embedding(vocab_size, embeding_vector_size,batch_input_shape=(batch_size, num_steps),trainable=True,name="embedding_vocab")  

In [13]:
# Define where to get the data for our embeddings from
inputs = embedding_layer(_input_data)
inputs

<tf.Tensor: shape=(30, 20, 200), dtype=float32, numpy=
array([[[-0.00963352,  0.04447431,  0.03000686, ..., -0.01430992,
         -0.04179025, -0.03740842],
        [-0.00185136,  0.0061741 , -0.02399608, ...,  0.00052229,
          0.02421384, -0.03178833],
        [ 0.03165312, -0.03180691, -0.02924181, ..., -0.04003506,
          0.04339501,  0.00341809],
        ...,
        [ 0.00384145, -0.025701  , -0.03223218, ..., -0.04989583,
         -0.04297003,  0.03399796],
        [ 0.04402603,  0.01031557, -0.04961705, ..., -0.04415311,
         -0.04264161, -0.04333409],
        [-0.04641173,  0.0193573 ,  0.03973095, ..., -0.01120675,
         -0.03314363, -0.02827821]],

       [[-0.03361372, -0.04295586,  0.03282306, ..., -0.04212505,
          0.03222534,  0.04298704],
        [ 0.0467957 ,  0.01119499, -0.03936114, ...,  0.01421765,
         -0.00408707,  0.00464406],
        [-0.00853928,  0.04816118,  0.03704181, ..., -0.04109078,
         -0.01007197,  0.0286814 ],
        ...,

<h3>Construyendo la red neuronal recurrente</h3>


Creamos la stacked LSTM usando <b>tf.keras.layers.StackedRNNCells</b>:


In [14]:
lstm_cell_l1 = tf.keras.layers.LSTMCell(hidden_size_l1)
lstm_cell_l2 = tf.keras.layers.LSTMCell(hidden_size_l2)

In [15]:
stacked_lstm = tf.keras.layers.StackedRNNCells([lstm_cell_l1, lstm_cell_l2])

<b>tf.keras.layers.RNN</b> crea una red recurrente usando<b>stacked_lstm</b>. 

La entrada debe ser un tensor de la forma: [batch_size, max_time, embedding_vector_size], en nuestro caso será (30, 20, 200)


In [16]:
layer  =  tf.keras.layers.RNN(stacked_lstm,[batch_size, num_steps],return_state=False,stateful=True,trainable=True)

También inicializamos los estados de la red:

<h4>_initial_state</h4>

Para cada LSTM, hay 2 matrices de estado, c_state y m_state. c_state y m_state representan "Cell state" y "Memory state". Cada capa oculta tiene un vector de tamaño 30, que mantiene los estados. Entonces, para 200 unidades escondidas en cada LSTM, tenemos una matriz de tamaño [30x200]


In [17]:
init_state = tf.Variable(tf.zeros([batch_size,embeding_vector_size]),trainable=False)

In [18]:
layer.inital_state = init_state

In [19]:
layer.inital_state

<tf.Variable 'Variable:0' shape=(30, 200) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

Miremos las salidas. La salida de stackedLSTM viene de 128 hidden_layer, y en cada paso de tiempo (=20), uno de ellos se activa. Usamos la activación lineal para mapear las 128 capas ocultas a una matriz  [30X20]


In [20]:
outputs = layer(inputs)

In [21]:
outputs

<tf.Tensor: shape=(30, 20, 128), dtype=float32, numpy=
array([[[-1.15399947e-03, -5.68167306e-04, -6.36982557e-04, ...,
         -4.73892607e-04,  5.77440369e-04,  9.37731005e-04],
        [-2.25645606e-03, -4.12659574e-04, -1.41658777e-04, ...,
         -2.64253211e-03, -4.77828173e-04,  2.03687442e-03],
        [-3.47123714e-03, -1.35197095e-03,  1.65716890e-04, ...,
         -3.61537002e-03, -4.11159941e-04,  1.61307643e-03],
        ...,
        [-8.39538034e-03, -2.45015253e-04, -1.09094812e-03, ...,
          1.08035376e-04, -5.07893157e-04,  2.33715540e-03],
        [-7.56491860e-03, -1.81483256e-03, -1.33696629e-03, ...,
          2.04627588e-03,  3.20677907e-04,  1.67608715e-03],
        [-6.53917482e-03, -2.30778474e-03, -2.84525519e-03, ...,
          4.18785587e-03,  6.00512576e-05, -1.63874269e-04]],

       [[ 1.09756053e-04,  1.32923410e-03,  1.75143490e-04, ...,
         -4.65967751e-04,  1.16835954e-03, -1.53120316e-03],
        [ 3.86477594e-04,  1.76512927e-03,  1.25

<h2>Capa densa</h2>
Creamos una capa densamente conectada que remodelará las salidas del tensor de  [30 x 20 x 128] a [30 x 20 x 10000].


In [22]:
dense = tf.keras.layers.Dense(vocab_size)

In [23]:
logits_outputs  = dense(outputs)

In [30]:
print("shape of the output from dense layer: ", logits_outputs.shape) #(batch_size, sequence_length, vocab_size)

shape of the output from dense layer:  (30, 20, 10000)


<h2>Capa de activación</h2>
Usamos softmax para obetner las probabilidades de que la salida esté en una de las posibilidades (10000 en este caso).


In [26]:
activation = tf.keras.layers.Activation('softmax')

In [27]:
output_words_prob = activation(logits_outputs)

In [31]:
print("shape of the output from the activation layer: ", output_words_prob.shape) #(batch_size, sequence_length, vocab_size)

shape of the output from the activation layer:  (30, 20, 10000)


Veamos la probabilidad de observar palabras para t = 0 hasta = 20:


In [37]:
print("The probability of observing words in t=0 to t=20", output_words_prob[0,0:num_steps])

The probability of observing words in t=0 to t=20 tf.Tensor(
[[9.99780095e-05 9.99922995e-05 9.99947006e-05 ... 1.00020879e-04
  9.99827971e-05 9.99914919e-05]
 [9.99729527e-05 9.99912663e-05 9.99962358e-05 ... 1.00030331e-04
  9.99581753e-05 9.99895565e-05]
 [9.99803524e-05 9.99943295e-05 1.00012709e-04 ... 1.00035977e-04
  9.99577169e-05 9.99934273e-05]
 ...
 [9.99300610e-05 1.00030367e-04 1.00004378e-04 ... 1.00011341e-04
  9.99942495e-05 9.98878968e-05]
 [9.99065378e-05 1.00010395e-04 9.99664844e-05 ... 1.00023492e-04
  9.99747208e-05 9.98966716e-05]
 [9.98915930e-05 1.00003337e-04 9.99345066e-05 ... 1.00023390e-04
  9.99777913e-05 9.98746909e-05]], shape=(20, 10000), dtype=float32)


<h3>Predicción</h3>
Cuál es la palabra que corresponde a la probabilidad de salida? Usemos la probabilidad máxima.


In [38]:
np.argmax(output_words_prob[0,0:num_steps], axis=1)

array([1464, 1137, 9909, 8233, 8233, 8233, 9604, 1260,  976,  976, 7646,
       7129, 7366, 7366, 7366, 7366, 1732, 1732, 1732, 1732])

Cuál es la verdad para la primer palabra de la primer sentencia? Puede obtenerla del tensor objetivo.


In [39]:
_targets[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

<h4>Función objetivo</h4>

Cuán similares son las palabras predichas a las objetivo?

Debemos definir nuestra función objetivo, para calcular la similaridad de los valores predecidos con la verdad y luego, penalizar el modelo con el error. Nuestro objetivo es minimizar la función de pérdida, que en este caso es la probabilidad logarítmica promedio negativa de las palabras objetivo:

$$\text{loss} = -\frac{1}{N}\sum_{i=1}^{N} \ln p_{\text{target}_i}$$

Esta función está disponible en TensorFlow a través de _tf.keras.losses.sparse_categorical_crossentropy_. Calcula la pérdida de entropía cruzada categórica para logits y la secuencia objetivo.

Los argumentos de la función son:  

<ul>
    <li>logits: Lista de tensores 2D de la forma [batch_size x num_decoder_symbols].</li>  
    <li>targets: Lista de tensores 1d batch-sized int32 del mismo largo que logits</li>   
</ul>


In [40]:
def crossentropy(y_true, y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

In [41]:
loss  = crossentropy(_targets, output_words_prob)

Miremos los 10 primeros valores de la pérdida:


In [42]:
loss[0,:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([9.2101345, 9.210351 , 9.209828 , 9.210473 , 9.210363 , 9.209582 ,
       9.209699 , 9.210181 , 9.210007 , 9.210093 ], dtype=float32)>

Definimos el costo como el promedio de las pérdidas:


In [43]:
cost = tf.reduce_sum(loss / batch_size)
cost

<tf.Tensor: shape=(), dtype=float32, numpy=184.20605>

<h3>Entrenamiento</h3>

Para el entrenamiento, seguimos los pasos siguientes:

<ol>
    <li>Definimos el optimizador</li>
    <li>Ensamblamos capas para construir el modelo</li>
    <li>Calculamos los gradientes basados en la función de pérdida.</li>
    <li>Aplicamos el optimizador a la tupla variables/gradientes.</li>
</ol>


<h4>1. Definimos el optimizador</h4>


In [44]:
# Create a variable for the learning rate
lr = tf.Variable(0.0, trainable=False)
optimizer = tf.keras.optimizers.SGD(lr=lr, clipnorm=max_grad_norm)

<h4>2. Ensamblamos capas para construir el modelo</h4>


In [45]:
model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(layer)
model.add(dense)
model.add(activation)
model.compile(loss=crossentropy, optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_vocab (Embedding)  (30, 20, 200)             2000000   
_________________________________________________________________
rnn (RNN)                    (30, 20, 128)             671088    
_________________________________________________________________
dense (Dense)                (30, 20, 10000)           1290000   
_________________________________________________________________
activation (Activation)      (30, 20, 10000)           0         
Total params: 3,961,088
Trainable params: 3,955,088
Non-trainable params: 6,000
_________________________________________________________________


<h4>2. Variables entrenables</h4>


Al definir una variable, si se pasa <i>trainable=True</i>, el constructor de la variable automáticamente agrega nuevas variables a la colección de grafos <b>GraphKeys.TRAINABLE_VARIABLES</b>. Ahora, usando <i>tf.trainable_variables()</i> puede obtener todas las variables creadas con trainable=True.


In [46]:
# Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
tvars = model.trainable_variables

Nota: Podemos encontrar el nombre y alcance de todas las variables:


In [47]:
[v.name for v in tvars] 

['embedding_vocab/embeddings:0',
 'rnn/stacked_rnn_cells/lstm_cell/kernel:0',
 'rnn/stacked_rnn_cells/lstm_cell/recurrent_kernel:0',
 'rnn/stacked_rnn_cells/lstm_cell/bias:0',
 'rnn/stacked_rnn_cells/lstm_cell_1/kernel:0',
 'rnn/stacked_rnn_cells/lstm_cell_1/recurrent_kernel:0',
 'rnn/stacked_rnn_cells/lstm_cell_1/bias:0',
 'dense/kernel:0',
 'dense/bias:0']

<h4>3. Calculamos los gradientes basados en la función de pérdida</h4>


**Gradiente**: El gradiente de una función representa la tasa de cambio de la función. Es un vector que apunta en la dirección de mayor crecimiento de la función y se calcula usando la operación derivada:


Comencemos con un ejemplo sencillo:
$$ z = \left(2x^2 + 3xy\right)$$


In [48]:
x = tf.constant(1.0)
y =  tf.constant(2.0)
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    g.watch(y)
    func_test = 2 * x * x + 3 * x * y

La función <b>tf.gradients()</b> le permite computar el gradiente simbólico de un tensor respecto a uno o más tensores. <b>tf.gradients(func, xs)</b>  construye derivadas parciales simbólicas de la suma de <b>func</b> w.r.t. <i>x</i> en <b>xs</b>.

Ahora miremos la derivada w.r.t. <b>var_x</b>:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2 + 3xy\right) = 4x + 3y $$


In [49]:
var_grad = g.gradient(func_test, x) # Will compute to 10.0
print(var_grad)

tf.Tensor(10.0, shape=(), dtype=float32)


la derivada w.r.t. <b>var_y</b>:
$$ \frac{\partial \:}{\partial \:y}\left(2x^2 + 3xy\right) = 3x $$


In [50]:
var_grad = g.gradient(func_test, y) # Will compute to 3.0
print(var_grad)

tf.Tensor(3.0, shape=(), dtype=float32)


Ahora vemos los gradientes w.r.t para todas las variables:


In [51]:
with tf.GradientTape() as tape:
    # Forward pass.
    output_words_prob = model(_input_data)
    # Loss value for this batch.
    loss  = crossentropy(_targets, output_words_prob)
    cost = tf.reduce_sum(loss,axis=0) / batch_size

In [52]:
# Get gradients of loss wrt the trainable variables.
grad_t_list = tape.gradient(cost, tvars)

In [53]:
print(grad_t_list)

[<tensorflow.python.framework.indexed_slices.IndexedSlices object at 0x7f049baed780>, <tf.Tensor: shape=(200, 1024), dtype=float32, numpy=
array([[-1.5256627e-07, -6.7794031e-07, -8.7616563e-08, ...,
         5.3887391e-07, -6.7084341e-07, -1.9336210e-07],
       [ 6.9934833e-07, -4.5459316e-07, -7.6316510e-08, ...,
        -1.2728367e-07,  4.6999193e-07,  4.3857995e-08],
       [ 2.9738970e-08,  1.4588701e-07,  5.3057556e-07, ...,
         2.6987630e-07,  3.5652761e-09, -4.3176478e-08],
       ...,
       [-4.5120191e-07, -6.6480175e-07, -1.1827770e-07, ...,
         6.3660934e-07,  1.6707975e-07, -1.4780258e-07],
       [ 1.1878119e-06,  1.8076659e-07,  8.1836511e-08, ...,
        -5.5646012e-08, -2.8694140e-07,  1.2399234e-07],
       [ 8.6045688e-08,  1.0653308e-06, -2.0499257e-07, ...,
        -5.7437262e-07, -3.3745656e-07, -4.2511931e-07]], dtype=float32)>, <tf.Tensor: shape=(256, 1024), dtype=float32, numpy=
array([[ 4.2339448e-08, -1.3613645e-07, -1.1695782e-09, ...,
         

ahora tenemos una lista de tensores t-list. Podemos usarlo para encontrar tensores recortados (clipped tensors). <b>clip_by_global_norm</b> recorta valores de múltiples tensores mediante el cociente de la suma de sus normas:


<b>clip_by_global_norm</b> tiene <i>t-list</i> como entrada y retorna 2 cosas:

<ul>
    <li>una lista de tensores recortados llamada <i>list_clipped</i></li> 
    <li>la norma global (global_norm) de todos los tensores en t_list</li> 
</ul>


In [54]:
# Define the gradient clipping threshold
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

 <tf.Tensor: shape=(200, 1024), dtype=float32, numpy=
 array([[-1.5256627e-07, -6.7794031e-07, -8.7616563e-08, ...,
          5.3887391e-07, -6.7084341e-07, -1.9336210e-07],
        [ 6.9934833e-07, -4.5459316e-07, -7.6316510e-08, ...,
         -1.2728367e-07,  4.6999193e-07,  4.3857995e-08],
        [ 2.9738970e-08,  1.4588701e-07,  5.3057556e-07, ...,
          2.6987630e-07,  3.5652761e-09, -4.3176478e-08],
        ...,
        [-4.5120191e-07, -6.6480175e-07, -1.1827770e-07, ...,
          6.3660934e-07,  1.6707975e-07, -1.4780258e-07],
        [ 1.1878119e-06,  1.8076659e-07,  8.1836511e-08, ...,
         -5.5646012e-08, -2.8694140e-07,  1.2399234e-07],
        [ 8.6045688e-08,  1.0653308e-06, -2.0499257e-07, ...,
         -5.7437262e-07, -3.3745656e-07, -4.2511931e-07]], dtype=float32)>,
 <tf.Tensor: shape=(256, 1024), dtype=float32, numpy=
 array([[ 4.2339448e-08, -1.3613645e-07, -1.1695782e-09, ...,
          1.7090683e-07,  6.7923679e-08,  4.3531614e-09],
        [ 8.9228593e-

<h4> 4. Aplicamos el optimizador a la tupla variables/gradientes. </h4>


In [55]:
# Create the training TensorFlow Operation through our optimizer
train_op = optimizer.apply_gradients(zip(grads, tvars))

<a id="ltsm"></a>

<h2>LSTM</h2>


Aprendimos a construir el modelo paso a paso. Ahora creemos una clase que represente el modelo.La clase necesita algunas cosas:


<ul>
    <li> Debemos crear el modelo de acuerdo a nuestros hiperparámetros definidos</li>
    <li> Debemos crear la estructura de la celda LSTM y conectarla con nuestra estructura RNN</li>
    <li> Debemos crear el word embeddings y apuntarlo a los datos de entrada</li>
    <li> Debemos crear la estructura de entrada para nuestra RNN</li>
    <li> Debemos crear una estructura logística para devolver las probabilidades de nuestras palabras</li>
    <li> Debemos crear las funciones de pérdida y costo para que nuestro optimizador funcione y luego crear el optimizador</li>
    <li> Debemos crear una operación de entrenamiento que pueda ejecutarse para entrenar nuestro modelo</li>
</ul>


In [3]:
class PTBModel(object):


    def __init__(self):
        ######################################
        # Setting parameters for ease of use #
        ######################################
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.hidden_size_l1 = hidden_size_l1
        self.hidden_size_l2 = hidden_size_l2
        self.vocab_size = vocab_size
        self.embeding_vector_size = embeding_vector_size
        # Create a variable for the learning rate
        self._lr = 1.0
        
        ###############################################################################
        # Initializing the model using keras Sequential API  #
        ###############################################################################
        
        self._model = tf.keras.models.Sequential()
        
        ####################################################################
        # Creating the word embeddings layer and adding it to the sequence #
        ####################################################################
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            self._embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embeding_vector_size,batch_input_shape=(self.batch_size, self.num_steps),trainable=True,name="embedding_vocab")  #[10000x200]
            self._model.add(self._embedding_layer)
            

        ##########################################################################
        # Creating the LSTM cell structure and connect it with the RNN structure #
        ##########################################################################
        # Create the LSTM Cells. 
        # This creates only the structure for the LSTM and has to be associated with a RNN unit still.
        # The argument  of LSTMCell is size of hidden layer, that is, the number of hidden units of the LSTM (inside A). 
        # LSTM cell processes one word at a time and computes probabilities of the possible continuations of the sentence.
        lstm_cell_l1 = tf.keras.layers.LSTMCell(hidden_size_l1)
        lstm_cell_l2 = tf.keras.layers.LSTMCell(hidden_size_l2)
        

        
        # By taking in the LSTM cells as parameters, the StackedRNNCells function junctions the LSTM units to the RNN units.
        # RNN cell composed sequentially of stacked simple cells.
        stacked_lstm = tf.keras.layers.StackedRNNCells([lstm_cell_l1, lstm_cell_l2])


        

        ############################################
        # Creating the input structure for our RNN #
        ############################################
        # Input structure is 20x[30x200]
        # Considering each word is represended by a 200 dimentional vector, and we have 30 batchs, we create 30 word-vectors of size [30xx2000]
        # The input structure is fed from the embeddings, which are filled in by the input data
        # Feeding a batch of b sentences to a RNN:
        # In step 1,  first word of each of the b sentences (in a batch) is input in parallel.  
        # In step 2,  second word of each of the b sentences is input in parallel. 
        # The parallelism is only for efficiency.  
        # Each sentence in a batch is handled in parallel, but the network sees one word of a sentence at a time and does the computations accordingly. 
        # All the computations involving the words of all sentences in a batch at a given time step are done in parallel. 

        ########################################################################################################
        # Instantiating our RNN model and setting stateful to True to feed forward the state to the next layer #
        ########################################################################################################
        
        self._RNNlayer  =  tf.keras.layers.RNN(stacked_lstm,[batch_size, num_steps],return_state=False,stateful=True,trainable=True)
        
        # Define the initial state, i.e., the model state for the very first data point
        # It initialize the state of the LSTM memory. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word.
        self._initial_state = tf.Variable(tf.zeros([batch_size,embeding_vector_size]),trainable=False)
        self._RNNlayer.inital_state = self._initial_state
    
        ############################################
        # Adding RNN layer to keras sequential API #
        ############################################        
        self._model.add(self._RNNlayer)
        
        #self._model.add(tf.keras.layers.LSTM(hidden_size_l1,return_sequences=True,stateful=True))
        #self._model.add(tf.keras.layers.LSTM(hidden_size_l2,return_sequences=True))
        
        
        ####################################################################################################
        # Instantiating a Dense layer that connects the output to the vocab_size  and adding layer to model#
        ####################################################################################################
        self._dense = tf.keras.layers.Dense(self.vocab_size)
        self._model.add(self._dense)
 
        
        ####################################################################################################
        # Adding softmax activation layer and deriving probability to each class and adding layer to model #
        ####################################################################################################
        self._activation = tf.keras.layers.Activation('softmax')
        self._model.add(self._activation)

        ##########################################################
        # Instantiating the stochastic gradient decent optimizer #
        ########################################################## 
        self._optimizer = tf.keras.optimizers.SGD(lr=self._lr, clipnorm=max_grad_norm)
        
        
        ##############################################################################
        # Compiling and summarizing the model stacked using the keras sequential API #
        ##############################################################################
        self._model.compile(loss=self.crossentropy, optimizer=self._optimizer)
        self._model.summary()


    def crossentropy(self,y_true, y_pred):
        return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    def train_batch(self,_input_data,_targets):
        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = self._model.trainable_variables
        # Define the gradient clipping threshold
        with tf.GradientTape() as tape:
            # Forward pass.
            output_words_prob = self._model(_input_data)
            # Loss value for this batch.
            loss  = self.crossentropy(_targets, output_words_prob)
            # average across batch and reduce sum
            cost = tf.reduce_sum(loss/ self.batch_size)
        # Get gradients of loss wrt the trainable variables.
        grad_t_list = tape.gradient(cost, tvars)
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
        # Create the training TensorFlow Operation through our optimizer
        train_op = self._optimizer.apply_gradients(zip(grads, tvars))
        return cost
        
    def test_batch(self,_input_data,_targets):
        #################################################
        # Creating the Testing Operation for our Model #
        #################################################
        output_words_prob = self._model(_input_data)
        loss  = self.crossentropy(_targets, output_words_prob)
        # average across batch and reduce sum
        cost = tf.reduce_sum(loss/ self.batch_size)

        return cost
    @classmethod
    def instance(cls) : 
        return PTBModel()

La estructura de la red está completa. Lo que falta es crear los métodos para la ejecución a través del tiempo, esto es un método run_epoch para ejecutarse en cada epoch y un script main que una todo.

run_epoch toma los datos de entrada y los alimenta a las operaciones relevantes. Este devolverá al final el resultado actual para la función de costo.


In [4]:

########################################################################################################################
# run_one_epoch takes as parameters  the model instance, the data to be fed, training or testing mode and verbose info #
########################################################################################################################
def run_one_epoch(m, data,is_training=True,verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.
    iters = 0
    
    m._model.reset_states()
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
        
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        #y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)
        if is_training : 
            loss=  m.train_batch(x, y)
        else :
            loss = m.test_batch(x, y)
                                   

        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += loss
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("Itr %d of %d, perplexity: %.3f speed: %.0f wps" % (step , epoch_size, np.exp(costs / iters), iters * m.batch_size / (time.time() - start_time)))
        


    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)


Ahora creamos el método main que une todo. El código aquí lee los datos desde un directorio, usando el módulo ayudante reader y luego entrena y evalúa el modelo en un subconjunto de datos tanto de testing como de validación.



In [11]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _, _ = raw_data

In [12]:
# Instantiates the PTBModel class
m=PTBModel.instance()   
K = tf.keras.backend 
for i in range(max_epoch):
    # Define the decay for this epoch
    lr_decay = decay ** max(i - max_epoch_decay_lr, 0.0)
    dcr = learning_rate * lr_decay
    m._lr = dcr
    K.set_value(m._model.optimizer.learning_rate,m._lr)
    print("Epoch %d : Learning rate: %.3f" % (i + 1, m._model.optimizer.learning_rate))
    # Run the loop for this epoch in the training mode
    train_perplexity = run_one_epoch(m, train_data,is_training=True,verbose=True)
    print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
    # Run the loop for this epoch in the validation mode
    valid_perplexity = run_one_epoch(m, valid_data,is_training=False,verbose=False)
    print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
# Run the loop in the testing mode to see how effective was our training
test_perplexity = run_one_epoch(m, test_data,is_training=False,verbose=False)
print("Test Perplexity: %.3f" % test_perplexity)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_vocab (Embedding)  (30, 20, 200)             2000000   
_________________________________________________________________
rnn_1 (RNN)                  (30, 20, 128)             671088    
_________________________________________________________________
dense_1 (Dense)              (30, 20, 10000)           1290000   
_________________________________________________________________
activation_1 (Activation)    (30, 20, 10000)           0         
Total params: 3,961,088
Trainable params: 3,955,088
Non-trainable params: 6,000
_________________________________________________________________
Epoch 1 : Learning rate: 1.000
Itr 10 of 1549, perplexity: 4715.241 speed: 1308 wps
Itr 164 of 1549, perplexity: 1093.509 speed: 1335 wps
Itr 318 of 1549, perplexity: 845.907 speed: 1330 wps
Itr 472 of 1549, perplexity: 699.792 speed: 132

Puede ver que la perplejidad de la red cae rápidamente luego de algunas iteraciones. Recuerde que a menor perplejidad mejor.

